In order to classify sentences with LSTM, I am going to use Many-to-one architecture from LSTM modules.

In [1]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.8MB 20.0MB/s 
     |████████████████████████████████| 102kB 14.8MB/s 
     |████████████████████████████████| 133kB 54.7MB/s 
     |████████████████████████████████| 174kB 47.2MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=e5e4bf4e4b850b0923fcef84e399b3204f220b6d40990f546f9fd9e6a27796c7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=cc07df97801ce20c841fd650a93079b9aa45430fa693377d80c3012b38c87910
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: 

In [2]:
CUDA_LAUNCH_BLOCKING=1

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data,datasets
import torchtext.legacy
import torch.optim as optim
import wandb
import time

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Parameters for Wandb

In [4]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
        'batch_size': {
            'values': [128, 64]
        },
        'learning_rate': {
            'values': [5e-2, 1e-3]
        }
    }
}

In [5]:
sweep_id = wandb.sweep(sweep_config, project="e216642_Assigment_WnB")

Create sweep with ID: kbosc5oy
Sweep URL: https://wandb.ai/icgencel/e216642_Assigment_WnB/sweeps/kbosc5oy


Definition of Tokenizer

In [6]:
def tokenizer(text):
  text = text.split(" ")
  return text

TEXT and LABEL creation

In [7]:
# In this part, basic tokenizer ais used and stop words are removed.
TEXT = data.Field(tokenize = tokenizer, batch_first= True, stop_words=stopwords.words('english'))
LABEL = data.LabelField(dtype = torch.float)

data is splitted to test and train sets

In [8]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 29.2MB/s]


Maximum and average sentence lengths are found

In [9]:
max_size=0  ## this part of the code find maximum length of the network
count=0
sum= 0
for i in  range(len(train_data)):
  if max_size < len(train_data[i].text):
    max_size =len(train_data[i].text)
    print(max_size)
  count +=1
  sum +=len(train_data[i].text)
print("average: ", sum/count)

202
211
272
288
299
428
477
522
592
642
1001
1035
1532
average:  137.90252


#Splitting data as train,valid, test data

In [10]:
TEXT = torchtext.legacy.data.Field(tokenize=tokenizer, batch_first=True,fix_length= 137 # it is filled with average size 
                            , stop_words=stopwords.words('english')) # preprocessing parameters can be used to add aditional  preprocessing steps
LABEL = torchtext.legacy.data.LabelField(dtype = torch.float)
train_data, test_data = torchtext.legacy.datasets.IMDB.splits(TEXT, LABEL) 
test_data, valid_data = test_data.split()

The following cell is taken from Lab5

In [11]:
# Build vocabularies
TEXT.build_vocab(train_data, 
                 max_size = 30000, # Select only the most important 30000 words
                 # Load pretrained embeddings
                 vectors = "glove.6B.100d", 
                 # Set unknown vectors
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
 99%|█████████▉| 397571/400000 [00:13<00:00, 29096.11it/s]

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creating Iterators

In [13]:
input_dim = len(TEXT.vocab)
embedding_dim = 100 # in order to create a emb vector with dimension 100
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
unk_idx = TEXT.vocab.stoi[TEXT.unk_token]

# LSTM Network

In [14]:
# In this part, 1-layer LSTM network is created. Before LSTM, embedding layer is used
# with dimension 100. At the output of LSTM, there is a dropout layer with prob 0.5
# Finally there is a linear layer, at the output

class LSTMNetwork(nn.Module):

  def __init__(self,input_dim,embedding_dim,pad_idx):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
    self.lstm = nn.LSTM(input_size =embedding_dim, hidden_size= 100, batch_first = True)
    self.dropout = nn.Dropout(0.5)
    self.linear = nn.Linear(100,1)

  def forward(self,input_ids):
        x = self.embedding(input_ids)
        lstm_out, (ht, ct) = self.lstm(x)
        ht = self.dropout(ht)
        out = self.linear(ht[-1])
        return out 

# Optimizer and Loss function

In [15]:

# Helper functions
def accuracy(preds, y):
    """ Return accuracy per batch. """
    correct = (torch.round(torch.sigmoid(preds)) == y).float() 
    return correct.sum() / len(correct)

def epoch_time(start_time, end_time):
    '''Track training time. '''
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
  
def train(model, iterator, optimizer, criterion):
    '''Train the model with specified data, optimizer, and loss function. '''
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        # Reset the gradient to not use them in multiple passes 
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = accuracy(predictions, batch.label)
        
        # Backprop
        loss.backward()
        
        # Optimize the weights
        optimizer.step()
        
        # Record accuracy and loss
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    '''Evaluate model performance. '''
    epoch_loss = 0
    epoch_acc = 0
    
    # Turm off dropout while evaluating
    model.eval()
    
    # No need to backprop in eval
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            
            acc = accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Wandb Part

Wandb function is added in this function, which parameters will be used in grid search are indicated. There is also some functions inside wandb function which are taken from IS784 - Lab5

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def Wandbfunc():

  TEXT = torchtext.legacy.data.Field(tokenize=tokenizer, batch_first=True,fix_length= 137 # it is filled with average size 
                            , stop_words=stopwords.words('english')) # preprocessing parameters can be used to add aditional  preprocessing steps
  LABEL = torchtext.legacy.data.LabelField(dtype = torch.float)
  train_data, test_data = torchtext.legacy.datasets.IMDB.splits(TEXT, LABEL) 
  test_data, valid_data = test_data.split()
  TEXT.build_vocab(train_data, 
                  max_size = 30000, # Select only the most important 30000 words
                  # Load pretrained embeddings
                  vectors = "glove.6B.100d", 
                  # Set unknown vectors
                  unk_init = torch.Tensor.normal_)

  LABEL.build_vocab(train_data)

  input_dim = len(TEXT.vocab)
  embedding_dim = 100 # in order to create a emb vector with dimension 100
  pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
  unk_idx = TEXT.vocab.stoi[TEXT.unk_token]
  
  config_defaults = {
        'epochs': 1,
        'batch_size': 32,
        'learning_rate': 1e-3,
    }

  wandb.init(config=config_defaults)
  
  config = wandb.config
  
    # Model is created and weigths are copied 
  # Finally model is transferred to GPU
  model = LSTMNetwork(input_dim, embedding_dim, pad_idx)
  model.embedding.weight.data.copy_(TEXT.vocab.vectors)

  model.embedding.weight.data[unk_idx] = torch.zeros(embedding_dim)
  model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)
  model = model.to(device)
  
  # iterator
  train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
      (train_data, valid_data, test_data), 
      batch_size = config.batch_size, 
      device = device)
  # optim_loss
  optimizer = optim.Adam(model.parameters(), lr = config.learning_rate)
  criterion = nn.BCEWithLogitsLoss()
  model = model.to(device)
  criterion = criterion.to(device)



  # Training loop
  N_EPOCHS = config.epochs

  best_valid_loss = float('inf')
  val_loss = []
  val_acc = []
  tr_loss = []
  tr_acc = []

  for epoch in range(N_EPOCHS):
      
      # Calculate training time
      start_time = time.time()
      # Get epoch losses and accuracies 
      train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
      valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
      end_time = time.time()
      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      
      # Save training metrics
      val_loss.append(valid_loss)
      val_acc.append(valid_acc)
      tr_loss.append(train_loss)
      tr_acc.append(train_acc)
      
      print(f'Epoch: {epoch+1:2} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
  
  
  wandb.log({"Train_loss":train_loss})
  wandb.log({"Valid_loss":valid_loss}) 
  wandb.log({"Train_acc":train_acc})
  wandb.log({"Valid_acc":valid_acc}) 


  test_loss, test_acc = evaluate(model, test_iterator, criterion)
  wandb.log({"Test_loss":test_loss})
  wandb.log({"Test_acc":test_acc})

In [17]:
def Wandb(train_data,test_data,valid_data):
  print(1)
  config_defaults = {
        'epochs': 1,
        'batch_size': 32,
        'learning_rate': 1e-3,
    }

  wandb.init(config=config_defaults)
  print(3)
  config = wandb.config
  
    # Model is created and weigths are copied 
  # Finally model is transferred to GPU
  model = LSTMNetwork(input_dim, embedding_dim, pad_idx)
  model.embedding.weight.data.copy_(TEXT.vocab.vectors)

  model.embedding.weight.data[unk_idx] = torch.zeros(embedding_dim)
  model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)
  model = model.to(device)
  print(5)
  # iterator
  train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
      (train_data, valid_data, test_data), 
      batch_size = config.batch_size, 
      device = device)
  # optim_loss
  optimizer = optim.Adam(model.parameters(), lr = config.learning_rate)
  criterion = nn.BCEWithLogitsLoss()
  model = model.to(device)
  criterion = criterion.to(device)

  print(7)

  # Training loop
  N_EPOCHS = config.epochs

  best_valid_loss = float('inf')
  val_loss = []
  val_acc = []
  tr_loss = []
  tr_acc = []

  for epoch in range(N_EPOCHS):
      
      # Calculate training time
      start_time = time.time()
      # Get epoch losses and accuracies 
      train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
      valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
      end_time = time.time()
      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      
      # Save training metrics
      val_loss.append(valid_loss)
      val_acc.append(valid_acc)
      tr_loss.append(train_loss)
      tr_acc.append(train_acc)
      
      print(f'Epoch: {epoch+1:2} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
  
  
  wandb.log({"Train_loss":train_loss})
  wandb.log({"Valid_loss":valid_loss}) 
  wandb.log({"Train_acc":train_acc})
  wandb.log({"Valid_acc":valid_acc}) 


  test_loss, test_acc = evaluate(model, test_iterator, criterion)
  wandb.log({"Test_loss":test_loss})
  wandb.log({"Test_acc":test_acc})

In [18]:
wandb.agent(sweep_id, function=Wandbfunc)

wandb: Agent Starting Run: krrt2x1f with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	learning_rate: 0.05
wandb: Currently logged in as: icgencel (use `wandb login --relogin` to force relogin)


 99%|█████████▉| 397571/400000 [00:30<00:00, 29096.11it/s]

Epoch:  1 | Epoch Time: 0m 4s
	Train Loss: 0.713 | Train Acc: 50.15%
	 Val. Loss: 0.691 |  Val. Acc: 50.99%
Epoch:  2 | Epoch Time: 0m 3s
	Train Loss: 0.632 | Train Acc: 66.09%
	 Val. Loss: 0.626 |  Val. Acc: 67.29%
Epoch:  3 | Epoch Time: 0m 3s
	Train Loss: 0.557 | Train Acc: 73.63%
	 Val. Loss: 0.634 |  Val. Acc: 67.09%
Epoch:  4 | Epoch Time: 0m 3s
	Train Loss: 0.524 | Train Acc: 74.77%
	 Val. Loss: 0.660 |  Val. Acc: 67.40%
Epoch:  5 | Epoch Time: 0m 3s
	Train Loss: 0.478 | Train Acc: 78.14%
	 Val. Loss: 0.622 |  Val. Acc: 68.15%


Train_loss,0.47779
_runtime,32
_timestamp,1623616908
_step,5
Valid_loss,0.62193
Train_acc,0.78145
Valid_acc,0.68146
Test_loss,0.62195
Test_acc,0.68073


Train_loss,▁
_runtime,▁▁▁▁██
_timestamp,▁▁▁▁██
_step,▁▂▄▅▇█
Valid_loss,▁
Train_acc,▁
Valid_acc,▁
Test_loss,▁
Test_acc,▁


...

wandb: Agent Starting Run: fi78t0fm with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001


Epoch:  1 | Epoch Time: 0m 5s
	Train Loss: 0.689 | Train Acc: 53.28%
	 Val. Loss: 0.697 |  Val. Acc: 51.38%
Epoch:  2 | Epoch Time: 0m 5s
	Train Loss: 0.691 | Train Acc: 52.15%
	 Val. Loss: 0.691 |  Val. Acc: 52.44%
Epoch:  3 | Epoch Time: 0m 5s
	Train Loss: 0.681 | Train Acc: 56.64%
	 Val. Loss: 0.693 |  Val. Acc: 52.24%
Epoch:  4 | Epoch Time: 0m 5s
	Train Loss: 0.684 | Train Acc: 54.86%
	 Val. Loss: 0.680 |  Val. Acc: 56.33%
Epoch:  5 | Epoch Time: 0m 5s
	Train Loss: 0.667 | Train Acc: 57.72%
	 Val. Loss: 0.689 |  Val. Acc: 51.76%
Epoch:  6 | Epoch Time: 0m 5s
	Train Loss: 0.606 | Train Acc: 65.68%
	 Val. Loss: 0.578 |  Val. Acc: 73.08%
Epoch:  7 | Epoch Time: 0m 5s
	Train Loss: 0.615 | Train Acc: 62.87%
	 Val. Loss: 0.696 |  Val. Acc: 51.84%
Epoch:  8 | Epoch Time: 0m 5s
	Train Loss: 0.582 | Train Acc: 68.03%
	 Val. Loss: 0.709 |  Val. Acc: 51.57%
Epoch:  9 | Epoch Time: 0m 5s
	Train Loss: 0.591 | Train Acc: 65.32%
	 Val. Loss: 0.706 |  Val. Acc: 53.97%
Epoch: 10 | Epoch Time: 0m 5

Train_loss,0.57164
_runtime,57
_timestamp,1623617264
_step,5
Valid_loss,0.55547
Train_acc,0.70215
Valid_acc,0.74682
Test_loss,0.55356
Test_acc,0.74615


Train_loss,▁
_runtime,▁▁▁▁██
_timestamp,▁▁▁▁██
_step,▁▂▄▅▇█
Valid_loss,▁
Train_acc,▁
Valid_acc,▁
Test_loss,▁
Test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
